In [1]:
import os
import numpy as np
#data_path = '/home/cqm/codes/paper/data/opensmile/'
data_path = '/home/cqm/codes/paper/data/voice_features/'
label_file = '/home/cqm/codes/paper/data/IEMOCAP/IEMOCAP.txt'
label_dic = {}
emotion2idx = {}
emotion2idx['neu'] = 0
emotion2idx['hap'] = 1
emotion2idx['exc'] = 1
emotion2idx['ang'] = 2
emotion2idx['sad'] = 3
for line in open(label_file, 'r').readlines():
    cur_name, wav_file, cur_label = line.split()
    cur_name += '.fea'
    label_dic[cur_name]= emotion2idx[cur_label]

In [2]:
from tqdm import tqdm

X = []
y = []
val_X = []
val_y = []
wav_names = []
val_names = []
total_fea = 51
n_steps = 200
for line in tqdm(open('/home/cqm/codes/paper/data/f_counter.txt', 'r').readlines()):
    cur_x = []
    cur_name, fea_num = line.strip().split()
    for features in open(data_path+cur_name, 'r').readlines():
        features = features.strip().split()
#         if "nan" in features:
#             print(cur_name)
        cur_x.append([float(i) if i != "nan" else 0.0 for i in features])
#     print(cur_name)
#     print(np.asarray(cur_x).shape)
    if label_dic[cur_name] == 2 and int(fea_num)> n_steps:
        ran_idx = np.random.randint(n_steps/2, int(fea_num)-n_steps/2, 10)
        for tmp_idx in ran_idx:
            X.append([cur_x[j][tmp_idx-n_steps//2:tmp_idx+n_steps//2] for j in range(total_fea)])
            y.append(label_dic[cur_name])
            wav_names.append(cur_name)
    elif (label_dic[cur_name] == 1 or label_dic[cur_name] == 3) and int(fea_num)>n_steps:
        ran_idx = np.random.randint(n_steps/2, int(fea_num)-n_steps/2, 5)
        for tmp_idx in ran_idx:
            X.append([cur_x[j][tmp_idx-n_steps//2:tmp_idx+n_steps//2] for j in range(total_fea)])
            y.append(label_dic[cur_name])
            wav_names.append(cur_name)
    else:
        for i in range(int(fea_num)//n_steps):
            X.append([cur_x[j][n_steps*i:n_steps*(i+1)] for j in range(total_fea)])
            y.append(label_dic[cur_name])
            wav_names.append(cur_name)
        if int(fea_num) >= n_steps and int(fea_num)%n_steps > 30:
            X.append([cur_x[j][-n_steps:] for j in range(total_fea)])
            y.append(label_dic[cur_name])
            wav_names.append(cur_name)
            
    if int(fea_num) < n_steps:
        zeros = np.zeros((total_fea, n_steps-int(fea_num)))
        cur_x = np.hstack((cur_x, zeros))
        X.append(cur_x)
        y.append(label_dic[cur_name])
        wav_names.append(cur_name)
        val_X.append(cur_x)
        val_y.append(label_dic[cur_name])
        val_names.append(cur_name)

    if int(fea_num)>= n_steps:
        val_X.append([cur_x[j][int(fea_num)//2-n_steps//2:int(fea_num)//2+n_steps//2] for j in range(total_fea)])
        val_y.append(label_dic[cur_name])
        val_names.append(cur_name)
#         for i in range(int(fea_num)//200):
#             X.append([cur_x[j][200*i:200*(i+1)] for j in range(total_fea)])
#             y.append(label_dic[cur_name])
#             wav_names.append(cur_name)
#         if int(fea_num) >= 200 and int(fea_num)%200 > 10:
#             X.append([cur_x[j][-200:] for j in range(total_fea)])
#             y.append(label_dic[cur_name])
#             wav_names.append(cur_name)

np.array(X).max()
np.array(X).shape

100%|██████████| 2942/2942 [00:34<00:00, 85.61it/s] 


(10226, 51, 200)

In [3]:
emotions = [0, 0, 0, 0]
for i in range(len(y)):
    emotions[y[i]] += 1
print(emotions)

[2157, 3287, 2314, 2468]


In [4]:
def get_ses(X, y, wav_names, ses_num):
    ses_x = []
    ses_y = []
    for idx, wav_name in enumerate(wav_names):
        assert type(wav_name[4]) == type(ses_num)
        if wav_name[4] == ses_num:
            ses_x.append(X[idx])
            ses_y.append(y[idx])
    ses_x = np.asarray(ses_x)
    ses_y = np.asarray(ses_y)
    data_min = np.min(ses_x, 0)
    data_max = np.max(ses_x, 0)
#     data_mean = np.mean(ses_x)
#     data_std = np.std(ses_x)
    ses_x = (ses_x - data_min) / (data_max - data_min)
    return ses_x, ses_y, data_min, data_max

In [5]:
def get_val(X, y, wav_names, ses_num, data_min, data_max):
    ses_x = []
    ses_y = []
    for idx, wav_name in enumerate(wav_names):
        assert type(wav_name[4]) == type(ses_num)
        if wav_name[4] == ses_num:
            ses_x.append(X[idx])
            ses_y.append(y[idx])
    ses_x = np.asarray(ses_x)
    ses_y = np.asarray(ses_y)
    ses_x = (ses_x - data_min) / (data_max - data_min)
    return ses_x, ses_y

In [13]:
import tensorflow as tf
from tensorflow.contrib import rnn
import numpy as np
import os

lr = 1e-3
bsize = 64
n_input = total_fea
n_hidden = 64
phase = tf.placeholder(tf.bool, [1], name='phase')
x = tf.placeholder(tf.float32, [None, total_fea, n_steps, 1])
y_ = tf.placeholder(tf.int64, [None])
keep_prob = tf.placeholder(tf.float32) 

def bilstm(x):
    x = tf.reshape(x,[-1, total_fea, n_steps])
    x = tf.unstack(x, axis = 2)
    
    lstm_fw_cell = rnn.BasicLSTMCell(n_hidden, forget_bias = 1.0)
    lstm_bw_cell = rnn.BasicLSTMCell(n_hidden, forget_bias = 1.0)
    
    outputs, fw_states, bw_states = rnn.static_bidirectional_rnn(lstm_fw_cell, lstm_bw_cell, x, dtype=tf.float32)
    outputs = tf.stack(outputs)
    outputs = tf.transpose(outputs, [1,0,2])
    
    u = tf.layers.dense(outputs, 1)
    u = tf.reshape(u, [-1, n_steps])
    attention = tf.nn.softmax(u)[:,:,None]
    out = tf.multiply(attention, outputs)
    out = tf.reduce_sum(out, axis=1)
#     out = tf.nn.dropout(out, 0.5)
    return out     

with tf.variable_scope("model2", reuse=tf.AUTO_REUSE):
    h_conv1 = tf.layers.conv2d(x, 64, [1, 6], [1, 1], padding="SAME")#bsize*23*500*64
    h_conv1 = tf.nn.max_pool(h_conv1, [1, 1, 4, 1], [1, 1, 4, 1], "VALID")#bsize*23*100*64
    h_conv2 = tf.layers.conv2d(h_conv1, 128, [total_fea, 2], [total_fea, 1], padding="SAME")#bsize*1*125*128
    h_conv2 = tf.nn.max_pool(h_conv2, [1, 1, 2, 1], [1, 1, 2, 1], "SAME")#bsize*1*25*128
    print(h_conv2)
#     h_conv3 = tf.layers.conv2d(h_conv2, 256, [1, 3], [1, 1], padding="SAME")#bsize*1*25*256
#     h_conv3 = tf.nn.max_pool(h_conv3, [1, 1, 2, 1], [1, 1, 2, 1], "VALID")#bsize*1*25*256
#     print h_conv3
#     h_conv3 = tf.layers.conv2d(h_conv1, 64, [1, 3], [1, 1], padding="VALID")
#     h_conv2 = tf.nn.max_pool(h_conv2, [1, 1, 2, 1], [1, 1, 2, 1], "VALID")
#     h_conv3= tf.layers.conv2d(h_conv2, 128, [1, 5], [1, 1], padding="VALID")
#     h_pool1 = tf.nn.avg_pool(h_conv3, [1, 1, n_steps-6, 1], [1, 1,n_steps-6, 1], "VALID")   
#     h_pool2 = tf.reshape(h_pool1,[-1, 64])
#     h_pool3 = tf.layers.dense(h_pool2, 128)
    h_conv2  = tf.reshape(h_conv2, [-1, 25, 128])
#     h_conv2 = tf.nn.dropout(h_conv2, keep_prob)

#     rnn_layers = tf.nn.rnn_cell.LSTMCell(48)
#     outputs, state = tf.nn.dynamic_rnn(rnn_layers, h_conv2, dtype=tf.float32)

    rnn_layers = [tf.nn.rnn_cell.LSTMCell(size) for size in [48, 64]]
#     initial_state = rnn_layers.zero_state(bsize, dtype=tf.float32)
#     outputs, state = tf.nn.dynamic_rnn(rnn_layers, h_conv2, initial_state=initial_state, dtype=tf.float32)
#     state = state.h
#     print('state', state)

    multi_rnn_cell = tf.nn.rnn_cell.MultiRNNCell(rnn_layers)
    cell_dr = tf.nn.rnn_cell.DropoutWrapper(multi_rnn_cell, output_keep_prob=keep_prob)
    lstm_out, state = tf.nn.dynamic_rnn(cell = cell_dr, inputs = h_conv2, dtype = tf.float32)
    state = tf.concat([state[0].h, state[1].h], axis=1)
    print(state)
    output_1 = tf.layers.dense(state, 64)
    output = tf.layers.dense(output_1, 4)

#     output = tf.layers.dense(state, 4)
    print(output)
#     b_out = bilstm(x)
#     h_con = tf.concat([h_pool2, b_out], 1)

#     h_con = tf.nn.dropout(h_con, keep_prob)
#     print("h_con" ,h_con)
    
#     h_fc1 = tf.layers.dense(h_con, 64)
#     h_fc2 = tf.layers.dense(h_fc1, 4)

    cross_entropy=tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y_,logits=output)
    train_step = tf.train.AdamOptimizer(lr).minimize(cross_entropy)
    correct_prediction = tf.equal(tf.argmax(output,1),y_)
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    

Tensor("model2/MaxPool_1:0", shape=(?, 1, 25, 128), dtype=float32)
Tensor("model2/concat:0", shape=(?, 112), dtype=float32)
Tensor("model2/dense_1/BiasAdd:0", shape=(?, 4), dtype=float32)


In [7]:
X1, y1, data_min_1, data_max_1 = get_ses(X, y, wav_names, '1')
X2, y2, data_min_2, data_max_2 = get_ses(X, y, wav_names, '2')
X3, y3, data_min_3, data_max_3 = get_ses(X, y, wav_names, '3')
X4, y4, data_min_4, data_max_4 = get_ses(X, y, wav_names, '4')
X5, y5, data_min_5, data_max_5 = get_ses(X, y, wav_names, '5')
Xs = [X1, X2, X3, X4, X5]
ys = [y1, y2, y3, y4, y5]
val_X1, val_y1 = get_val(val_X, val_y, val_names, '1', data_min_1, data_max_1)
val_X2, val_y2 = get_val(val_X, val_y, val_names, '2', data_min_2, data_max_2)
val_X3, val_y3 = get_val(val_X, val_y, val_names, '3', data_min_3, data_max_3)
val_X4, val_y4 = get_val(val_X, val_y, val_names, '4', data_min_4, data_max_4)
val_X5, val_y5 = get_val(val_X, val_y, val_names, '5', data_min_5, data_max_5)
val_Xs = [val_X1, val_X2, val_X3, val_X4, val_X5]
val_ys = [val_y1, val_y2, val_y3, val_y4, val_y5]
print("dataset has done.")

dataset has done.


In [14]:
if(os.path.exists('result_1.txt')):
    os.remove('result_1.txt')
    
    
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer()) 
n_epochs = 30
drop_out_keep_prob = 0.75
val_accuracy = []
for ses_idx in range(5):   
    val_set = val_Xs[ses_idx][:,:,:,None]
    print("val %d reshape has done."%ses_idx)
    val_label = val_ys[ses_idx]
    train_set = np.vstack([Xs[j] for j in range(5) if j != ses_idx])[:,:,:,None]
    print("train reshape has done.")
    train_label = np.concatenate([ys[j] for j in range(5) if j != ses_idx])
    print("data has already.")
    best_acc = 0.0
    count = 0
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(n_epochs):
        for i in range(len(train_set)//bsize):
            b_x = [] 
            b_y = []
            idx = np.random.choice(len(train_set),bsize,replace=False)
            b_x = train_set[idx]
            b_y = train_label[idx]
#             for j in range(bsize):
#                 b_x.append(train_set[idx[j]])
#                 b_y.append(train_label[idx[j]])
            if i%10 == 0 and i != 0:
                    train_accuracy = accuracy.eval(feed_dict={x:b_x, y_:b_y,keep_prob:1.0})
                    print('val %d epoch %d step %d, training accuracy %g'%(ses_idx, epoch, i, train_accuracy))
            train_step.run(feed_dict={x:b_x, y_:b_y, keep_prob: drop_out_keep_prob})
        try:
            train_accuracy = accuracy.eval(feed_dict={x:val_set,y_:val_label, keep_prob:1.0})
        except:
            train_accuracy1 = accuracy.eval(feed_dict={x:val_set[:len(val_set)//2],y_:val_label[:len(val_set)//2], keep_prob:1.0})
            train_accuracy2 = accuracy.eval(feed_dict={x:val_set[len(val_set)//2:],y_:val_label[len(val_set)//2:], keep_prob:1.0})
            train_accuracy = (train_accuracy1+train_accuracy2)//2
        print("Epoch {}'s val accuracy is {:.4f}" . format(epoch, train_accuracy))
        if best_acc < train_accuracy:
            best_acc = train_accuracy
            try:
                tmp_pre = output.eval(feed_dict={x:val_set,y_:val_label, keep_prob:1.0}).argmax(1)
            except:
                tmp_pre1 = output.eval(feed_dict={x:val_set[:len(val_set)//2],y_:val_label[:len(val_set)//2], keep_prob:1.0})
                tmp_pre2 = output.eval(feed_dict={x:val_set[len(val_set)//2:],y_:val_label[len(val_set)//2:], keep_prob:1.0})
                tmp_pre = np.vstack((tmp_pre1,tmp_pre2)).argmax(1)
            count = 0
        elif count < 5:
            count += 1
        else:
            break
    val_accuracy.append(best_acc)
    with open('result_1.txt', 'a') as fout:
        for t in range(len(val_set)):
            fout.writelines([str(tmp_pre[t]), '   ',str(val_label[t]), '\n'])
print(val_accuracy)

/home/zyq/anaconda3/envs/python3.5/lib/python3.5/site-packages/tensorflow/python/client/session.py:1645: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


val 0 reshape has done.
train reshape has done.
data has already.
val 0 epoch 0 step 10, training accuracy 0.421875
val 0 epoch 0 step 20, training accuracy 0.46875
val 0 epoch 0 step 30, training accuracy 0.40625
val 0 epoch 0 step 40, training accuracy 0.5625
val 0 epoch 0 step 50, training accuracy 0.453125
val 0 epoch 0 step 60, training accuracy 0.359375
val 0 epoch 0 step 70, training accuracy 0.40625
val 0 epoch 0 step 80, training accuracy 0.40625
val 0 epoch 0 step 90, training accuracy 0.578125
val 0 epoch 0 step 100, training accuracy 0.40625
val 0 epoch 0 step 110, training accuracy 0.578125
val 0 epoch 0 step 120, training accuracy 0.765625
Epoch 0's val accuracy is 0.5808
val 0 epoch 1 step 10, training accuracy 0.578125
val 0 epoch 1 step 20, training accuracy 0.578125
val 0 epoch 1 step 30, training accuracy 0.546875
val 0 epoch 1 step 40, training accuracy 0.484375
val 0 epoch 1 step 50, training accuracy 0.609375
val 0 epoch 1 step 60, training accuracy 0.578125
val 0

val 0 epoch 13 step 10, training accuracy 0.859375
val 0 epoch 13 step 20, training accuracy 0.921875
val 0 epoch 13 step 30, training accuracy 0.78125
val 0 epoch 13 step 40, training accuracy 0.796875
val 0 epoch 13 step 50, training accuracy 0.875
val 0 epoch 13 step 60, training accuracy 0.78125
val 0 epoch 13 step 70, training accuracy 0.9375
val 0 epoch 13 step 80, training accuracy 0.96875
val 0 epoch 13 step 90, training accuracy 0.84375
val 0 epoch 13 step 100, training accuracy 0.828125
val 0 epoch 13 step 110, training accuracy 0.859375
val 0 epoch 13 step 120, training accuracy 0.875
Epoch 13's val accuracy is 0.6558
val 1 reshape has done.
train reshape has done.
data has already.
val 1 epoch 0 step 10, training accuracy 0.3125
val 1 epoch 0 step 20, training accuracy 0.546875
val 1 epoch 0 step 30, training accuracy 0.578125
val 1 epoch 0 step 40, training accuracy 0.515625
val 1 epoch 0 step 50, training accuracy 0.46875
val 1 epoch 0 step 60, training accuracy 0.53125
v

val 2 epoch 1 step 20, training accuracy 0.546875
val 2 epoch 1 step 30, training accuracy 0.71875
val 2 epoch 1 step 40, training accuracy 0.578125
val 2 epoch 1 step 50, training accuracy 0.71875
val 2 epoch 1 step 60, training accuracy 0.625
val 2 epoch 1 step 70, training accuracy 0.671875
val 2 epoch 1 step 80, training accuracy 0.734375
val 2 epoch 1 step 90, training accuracy 0.6875
val 2 epoch 1 step 100, training accuracy 0.796875
val 2 epoch 1 step 110, training accuracy 0.75
val 2 epoch 1 step 120, training accuracy 0.671875
Epoch 1's val accuracy is 0.5853
val 2 epoch 2 step 10, training accuracy 0.734375
val 2 epoch 2 step 20, training accuracy 0.71875
val 2 epoch 2 step 30, training accuracy 0.65625
val 2 epoch 2 step 40, training accuracy 0.8125
val 2 epoch 2 step 50, training accuracy 0.59375
val 2 epoch 2 step 60, training accuracy 0.6875
val 2 epoch 2 step 70, training accuracy 0.6875
val 2 epoch 2 step 80, training accuracy 0.65625
val 2 epoch 2 step 90, training acc

val 3 epoch 3 step 20, training accuracy 0.75
val 3 epoch 3 step 30, training accuracy 0.65625
val 3 epoch 3 step 40, training accuracy 0.78125
val 3 epoch 3 step 50, training accuracy 0.75
val 3 epoch 3 step 60, training accuracy 0.640625
val 3 epoch 3 step 70, training accuracy 0.734375
val 3 epoch 3 step 80, training accuracy 0.765625
val 3 epoch 3 step 90, training accuracy 0.71875
val 3 epoch 3 step 100, training accuracy 0.765625
val 3 epoch 3 step 110, training accuracy 0.734375
val 3 epoch 3 step 120, training accuracy 0.78125
Epoch 3's val accuracy is 0.7303
val 3 epoch 4 step 10, training accuracy 0.6875
val 3 epoch 4 step 20, training accuracy 0.734375
val 3 epoch 4 step 30, training accuracy 0.875
val 3 epoch 4 step 40, training accuracy 0.84375
val 3 epoch 4 step 50, training accuracy 0.71875
val 3 epoch 4 step 60, training accuracy 0.75
val 3 epoch 4 step 70, training accuracy 0.796875
val 3 epoch 4 step 80, training accuracy 0.765625
val 3 epoch 4 step 90, training accur

val 4 epoch 7 step 30, training accuracy 0.828125
val 4 epoch 7 step 40, training accuracy 0.8125
val 4 epoch 7 step 50, training accuracy 0.953125
val 4 epoch 7 step 60, training accuracy 0.84375
val 4 epoch 7 step 70, training accuracy 0.828125
val 4 epoch 7 step 80, training accuracy 0.859375
val 4 epoch 7 step 90, training accuracy 0.8125
val 4 epoch 7 step 100, training accuracy 0.8125
val 4 epoch 7 step 110, training accuracy 0.90625
val 4 epoch 7 step 120, training accuracy 0.859375
Epoch 7's val accuracy is 0.6402
val 4 epoch 8 step 10, training accuracy 0.84375
val 4 epoch 8 step 20, training accuracy 0.71875
val 4 epoch 8 step 30, training accuracy 0.8125
val 4 epoch 8 step 40, training accuracy 0.859375
val 4 epoch 8 step 50, training accuracy 0.890625
val 4 epoch 8 step 60, training accuracy 0.765625
val 4 epoch 8 step 70, training accuracy 0.84375
val 4 epoch 8 step 80, training accuracy 0.875
val 4 epoch 8 step 90, training accuracy 0.875
val 4 epoch 8 step 100, training 

In [9]:
print(val_accuracy)

[0.69999999, 0.75283021, 0.65390748, 0.77902621, 0.68809849]


In [17]:
res = np.zeros((4, 4))
for line in open('/home/cqm/codes/paper/result_1.txt','r').readlines():
    line = line.strip().split()
    res[int(line[1])][int(line[0])] += 1
print(res)

[[ 789.   52.   82.  175.]
 [ 207.  669.   49.   22.]
 [  62.   20.  197.   10.]
 [ 157.   11.   20.  420.]]


In [15]:
print(np.array(val_accuracy).sum()/5)

0.709385871887


In [16]:
val_num = np.array([len(val_X1), len(val_X2),len(val_X3),len(val_X4),len(val_X5)])
print((val_accuracy*val_num).sum()/(val_num.sum()))

0.705302515235
